In [ ]:
import os
import cv2
import pickle
import numpy as np

In [ ]:
def wrap_images(img0, img1, geo_info, geom_type):
    rectified_image0 = img0
    rectified_image1 = None
    if geo_info is not None and len(geo_info) != 0:
        H = np.array(geo_info["Homography"])
        if geom_type == "Homography":
            rectified_image1 = cv2.warpPerspective(img1, H, (img0.shape[1], img0.shape[0]))
        else:
            print("Error: Unknown geometry type")
    return rectified_image0, rectified_image1

In [ ]:
def change_estimate_geom(input_image0, input_image1, matches_info, choice):
    if (matches_info is None) or (len(matches_info) < 1) or ("geom_info" not in matches_info.keys()):
        print(f'len(matches_info):{len(matches_info)}<1')
        return None, None
    geom_info = matches_info["geom_info"]
    rectified_image0, rectified_image1 = wrap_images(input_image0, input_image1, geom_info, choice)
    return rectified_image0, rectified_image1

In [ ]:
def compute_geom(mkpts0, mkpts1):
    if mkpts0 is not None and mkpts1 is not None:
        if len(mkpts0) < 2 * 4:
            return {}
        geo_info = {}
        H, _ = cv2.findHomography(mkpts1, mkpts0, method=cv2.USAC_MAGSAC, ransacReprojThreshold=8, confidence=0.999, maxIters=10000)
        if H is not None:
            geo_info["Homography"] = H.tolist()
        return geo_info
    else:
        return {}

In [ ]:
def make_matching_figure(img_clear, img_hazy, mkpts_clear, mkpts_clear_hazy, mkpts_syn=None, mkpts_syn_hazy=None):
    if mkpts_syn_hazy is not None and mkpts_syn_hazy is not None:
        if mkpts_syn.ndim != 1 and mkpts_syn_hazy.ndim != 1 and mkpts_clear.ndim != 1 and mkpts_clear_hazy.ndim != 1:
            mkpts_clear = np.concatenate((mkpts_syn, mkpts_clear), axis=0)
            mkpts_clear_hazy = np.concatenate((mkpts_syn_hazy, mkpts_clear_hazy), axis=0)

    geom_info = compute_geom(mkpts_clear, mkpts_clear_hazy)
    choice_estimate_geom = "Homography"
    if not geom_info:
        wrapped_image = img_hazy
    else:
        _, wrapped_image = change_estimate_geom(img_clear, img_hazy, {"geom_info": geom_info}, choice_estimate_geom)
    wrapped_image = cv2.cvtColor(wrapped_image, cv2.COLOR_RGB2BGR)
    return wrapped_image

In [ ]:
def main():
    hazy_clear_pkl_dir = ''  # (haze-free images and real-haze images) pkl folder
    hazy_syn_pkl_dir = ''  # (synthetic-haze images and real-haze images) pkl folder
    save_dir = ''
    os.makedirs(save_dir, exist_ok=True)

    for pkl_file in os.listdir(hazy_clear_pkl_dir):
        if pkl_file.endswith(".pkl"):
            pkl_clear_path = os.path.join(hazy_clear_pkl_dir, pkl_file)
            pkl_syn_path = os.path.join(hazy_syn_pkl_dir, pkl_file)
            with open(pkl_clear_path, "rb") as f:
                pred_clear = pickle.load(f)
            with open(pkl_syn_path, "rb") as f:
                pred_syn = pickle.load(f)

            img_clear = pred_clear["image0_orig"]
            img_hazy = pred_clear["image1_orig"]
            mkpts_clear = pred_clear["keypoints0_orig"]
            mkpts_clear_hazy = pred_clear["keypoints1_orig"]
            mkpts_syn = pred_syn["keypoints0_orig"]
            mkpts_syn_hazy = pred_syn["keypoints1_orig"]

            hazy_align_image = make_matching_figure(img_clear, img_hazy, mkpts_clear, mkpts_clear_hazy, mkpts_syn, mkpts_syn_hazy)
            save_path = os.path.join(save_dir, pkl_file[:-4] + ".jpg")
            cv2.imwrite(save_path, hazy_align_image)

In [ ]:
main()